# Object Detection in Pytorch document

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

import os

%matplotlib inline

## 경로 탐색

In [15]:
# 데이터 경로

data_path = 'data/PennFudanPed'
os.listdir(data_path)

['added-object-list.txt', 'Annotation', 'PedMasks', 'PNGImages', 'readme.txt']

In [19]:
# 이미지 경로
os.listdir(os.path.join(data_path,'PNGImages'))[:5]

['FudanPed00001.png',
 'FudanPed00002.png',
 'FudanPed00003.png',
 'FudanPed00004.png',
 'FudanPed00005.png']

In [22]:
# 마스크 경로
os.listdir(os.path.join(data_path, 'PedMasks'))[:5]

['FudanPed00001_mask.png',
 'FudanPed00002_mask.png',
 'FudanPed00003_mask.png',
 'FudanPed00004_mask.png',
 'FudanPed00005_mask.png']

## Dataset 생성

In [297]:
class PennFudanDataset(object):
    
    def __init__(self, root, transforms):
        self.root = data_path
        self.transforms = transforms
        
        self.imgs = sorted(os.listdir(os.path.join(data_path, 'PNGImages')))
        self.masks = sorted(os.listdir(os.path.join(data_path, 'PedMasks')))
        
    def __getitem__(self, idx):
        
        img_path = os.path.join(self.root, 'PNGImages', self.imgs[idx])
        mask_path = os.path.join(self.root, 'PedMasks', self.masks[idx])
        
        img = Image.open(img_path).convert('RGB')
        
        mask = Image.open(mask_path)
        
        mask = np.array(mask)
        
        obj_ids = np.unique(mask) # 결과로 0,1,2 값이 나옴
        
        obj_ids = obj_ids[1:] 
        # 0은 background을 의미하기 때문에 제외힌다(1,2)
                                
        masks = mask == obj_ids[:, None, None]
        # obj_ids는 1,2 값만 존재함(0은 제외했음)
        # ojb_ids[:, None, None]의 결과는 shape = (2, 1, 1)이 만들어짐
        # mask는 0,1,2로 이루어진 사진(W x H)이기 때문에
        # mask == ojb_ids 코드로 인해 (2 X W X H)로 변환된다(1이 True인 행렬, 2가 True인 행렬)
        # 각각 1과 2가 있는 부분은 True로 바뀌고 나머지는 False로 바뀐다(masking 작업)
        
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(mask[i]) # 값이 1인 부분의 index를 모두 찾는다(id 별로)
            xmin = np.min(pos[1]) # 행이 y축, 열이 x축이기 때문에 pos 결과의 index가 x=1, y=0이 된다.
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])